# Setup

## Pacotes

O pacote `vectordb2` do Python é utilizado para armazenar e recuperar textos usando técnicas de chunking (segmentação de texto), embeddings (conversão de texto para vetores numéricos) e busca vetorial.

In [7]:
%pip install vectordb2

In [ ]:
%pip install networkx

## Aquisição dos dados

Aqui é importado a lista de palavras do documento.

In [8]:
import requests

arquivo = requests.get('https://raw.githubusercontent.com/Cehiim/TeoriaDosGrafos/refs/heads/main/Projeto/palavras.txt').text

In [9]:
palavras = arquivo.split()

['Ecossistema', 'Sustentabilidade', 'Biodiversidade', 'Reciclagem', 'Conservação', 'Poluição', 'Desmatamento', 'Reflorestamento', 'Erosão', 'Compostagem', 'Biodegradável', 'Emissões', 'Pegada', 'Recursos', 'Preservação', 'Ecologia', 'Habitat', 'Fauna', 'Flora', 'Agroecologia', 'Bioma', 'Ciclo', 'Desenvolvimento', 'Economia', 'Efluentes', 'Gestão', 'Impacto', 'Mata', 'Amazônia', 'Cerrado', 'Pantanal', 'Biotecnologia', 'Agrofloresta', 'Agricultura', 'Aquicultura', 'Biocombustível', 'Solar', 'Eólica', 'Hidrelétrica', 'Resíduos', 'Saneamento', 'Tratamento', 'Uso', 'Zona', 'Proteção', 'Ambiental', 'Clima', 'Solo', 'Água', 'Floresta']


## Embedding das palavras e inserção na memória/banco de dados

Aqui é utilizado a biblioteca VectorDB para criar uma memória virtual.

- chunking_strategy: Este parâmetro define a estratégia de fragmentação dos dados. No caso, estamos usando a estratégia de janela deslizante ("sliding_window"). Isso significa que os dados serão divididos em pedaços (chunks) de tamanho fixo, especificado pelo parâmetro window_size.

- window_size: O tamanho dos chunks é definido como 1.

- overlap: O valor de sobreposição (overlap) é 0. Isso indica quantos elementos de sobreposição existirão entre os chunks adjacentes.

In [10]:
from vectordb import Memory

memory = Memory(chunking_strategy={"mode": "sliding_window", "window_size": 1, "overlap": 0})

Initiliazing embeddings:  normal
OK.


In [11]:
for i in range(50):
  memory.save(palavras[i])

# Sistema

Quanto menor é a distância, maior é a proximidade semântica.

In [12]:
for i in range(50):
  print(f"\n\nBusca: {palavras[i]}\n")
  buscas = memory.search(palavras[i], top_n=6)

  for j in range(1,6):
    palavra = buscas[j]['chunk']
    distancia = buscas[j]['distance']
    print(f"Palavra: {palavra}\nDistância: {distancia:.2f}\n")



Busca: Ecossistema

Palavra: Ecologia
Distância: 0.44

Palavra: Economia
Distância: 0.49

Palavra: Bioma
Distância: 0.58

Palavra: Biodiversidade
Distância: 0.58

Palavra: Sustentabilidade
Distância: 0.59



Busca: Sustentabilidade

Palavra: Biodiversidade
Distância: 0.45

Palavra: Economia
Distância: 0.56

Palavra: Ambiental
Distância: 0.56

Palavra: Ecossistema
Distância: 0.59

Palavra: Biotecnologia
Distância: 0.61



Busca: Biodiversidade

Palavra: Sustentabilidade
Distância: 0.45

Palavra: Agroecologia
Distância: 0.49

Palavra: Biotecnologia
Distância: 0.49

Palavra: Biodegradável
Distância: 0.53

Palavra: Bioma
Distância: 0.55



Busca: Reciclagem

Palavra: Compostagem
Distância: 0.56

Palavra: Reflorestamento
Distância: 0.64

Palavra: Recursos
Distância: 0.74

Palavra: Biotecnologia
Distância: 0.74

Palavra: Bioma
Distância: 0.76



Busca: Conservação

Palavra: Preservação
Distância: 0.24

Palavra: Proteção
Distância: 0.61

Palavra: Aquicultura
Distância: 0.77

Palavra: Poluiç